In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.optimizers import RMSprop
import numpy as np
import random
import sys

In [5]:
filename = "Trading with Mexico.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
print(raw_text[0:1000])



﻿the project gutenberg ebook of trading with mexico
    
this ebook is for the use of anyone anywhere in the united states and
most other parts of the world at no cost and with almost no restrictions
whatsoever. you may copy it, give it away or re-use it under the terms
of the project gutenberg license included with this ebook or online
at www.gutenberg.org. if you are not located in the united states,
you will have to check the laws of the country where you are located
before using this ebook.

title: trading with mexico

author: wallace thompson

release date: february 26, 2025 [ebook #75469]

language: english

original publication: new york: dodd, mead and company, 1921

credits: the online distributed proofreading team at https://www.pgdp.net (this file was produced from images generously made available by the internet archive)


*** start of the project gutenberg ebook trading with mexico ***





                              trading with
                                 mexico


In [6]:
raw_text = ''.join(c for c in raw_text if not c.isdigit())
# chars =  sorted(list(set(raw_text)))

In [7]:
import string

def clean_text(raw_text):
    # Create a translation table that maps every punctuation character to None
    translator = str.maketrans('', '', string.punctuation)
    # Also remove digits as per the original notebook logic, if desired:
    # translator = str.maketrans('', '', string.punctuation + string.digits)
    return raw_text.translate(translator)

# Apply the cleaning function
raw_text = clean_text(raw_text)

# Re-calculate chars and mapping (Cell 6 & 7 logic combined)
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters:", n_chars)
print("Total Vocab:", n_vocab)

Total Characters: 378429
Total Vocab: 45


In [8]:
char_to_int = dict((c,i) for i, c in enumerate(chars))
int_to_char = dict((i,c) for i, c in enumerate(chars))

In [9]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters in the text; corpus length:", n_chars)
print("Total Vocab", n_vocab)

Total Characters in the text; corpus length: 378429
Total Vocab 45


In [10]:
seq_length = 60
step = 10
sentences =[]
next_chars = []

for i in range(0, n_chars -seq_length, step):
    sentences.append(raw_text[i: i + seq_length])
    next_chars.append(raw_text[i + seq_length])

n_patterns = len(sentences)
print('Number of sequences:', n_patterns)

Number of sequences: 37837


In [11]:
x = np.zeros((len(sentences), seq_length, n_vocab), dtype=np.bool_)
y = np.zeros((len(sentences), n_vocab), dtype=np.bool_)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_to_int[char]] = 1
    y[i, char_to_int[next_chars[i]]] = 1

print(x.shape)
print(y.shape)
print(y[0:10])

(37837, 60, 45)
(37837, 45)
[[False False False False False False False False False False False False
  False False False False False False False False  True False False False
  False False False False False False False False False False False False
  False False False False False False False False False]
 [False  True False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False]
 [False False False False False False False False False False False False
  False False False False False False False False  True False False False
  False False False False False False False False False False False False
  False False False False False False False False False]
 [False False False False False False False False False False False False
  False False False  True False False False False False False

In [12]:
model = Sequential()
model.add(LSTM(128, input_shape=(seq_length, n_vocab)))
model.add(Dense(n_vocab, activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │        89,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 45)             │         5,805 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 94,893 (370.68 KB)

 Trainable params: 94,893 (370.68 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
from keras.callbacks import ModelCheckpoint

filepath = "save_weights-{epoch:02d}-{loss:.4f}.keras"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [checkpoint]

In [14]:
import numpy as np

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')

    # Avoid division by zero if temperature is extremely close to 0
    if temperature <= 1e-8:
        return np.argmax(preds)

    # Reweight the distribution using the temperature
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    # Draw from the reweighted distribution
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [15]:
# Configuration
start_epoch = 1
end_epoch = 60

# Iterate for the specific number of epochs
for epoch in range(start_epoch, end_epoch + 1):
    print(f'\\nEpoch {epoch}/{end_epoch}')

    # Fit the model for 1 epoch
    # Note: We assume 'x' and 'y' are already prepared from previous cells
    model.fit(x, y, batch_size=128, epochs=1)

    # Select a random seed text from the corpus
    start_index = random.randint(0, len(raw_text) - seq_length - 1)
    seed_text = raw_text[start_index: start_index + seq_length]
    print(f'--- Generating with seed: "{seed_text}"')

    # Iterate through the required temperatures
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print(f'------ temperature: {temperature}')
        sys.stdout.write(seed_text)

        # We need a copy of the seed so we don't corrupt it for the next temperature
        curr_text = seed_text

        # Generate 400 characters (or desired length)
        for i in range(400):
            # One-hot encode the current sequence
            sampled = np.zeros((1, seq_length, n_vocab))
            for t, char in enumerate(curr_text):
                sampled[0, t, char_to_int[char]] = 1.

            # Predict next character
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = int_to_char[next_index]

            # Append prediction and slide the window
            curr_text = curr_text[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print() # Print a newline after each temperature generation

\nEpoch 1/60
296/296 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 2.6872
--- Generating with seed: " advance over these figures in the nine
years since the fall"
------ temperature: 0.2
 advance over these figures in the nine
years since the fall the wor the wor the perite to wer the wor the wall wor and wor the wor the work the wore the rating of the pand in of the poun in and the wore wor the wand wor the wor the wand wat to the pare to to the pare the pond to the ment out wer the poun wor the ware the wer of the pand wor the wand the buct of the dere to the wand wor the wor the poun and the poppar wor the bene to the pore the pond the 
------ temperature: 0.5
 advance over these figures in the nine
years since the fall cor whive the pall und bat wor perice in the ralan buple the worcen wor hel wal hen derre to so the wor to the couns in ate ion cout han hat proper to the wating wer the hand or the mexico to somen of the hert the pond as pere out and in the pomen bun the and wal  and and

### Analysis of LSTM Text Generation at Varying Temperatures

The experiment involved training a Long Short-Term Memory (LSTM) network on the text "Trading with Mexico" over 60 epochs. The loss metric dropped significantly from approximately 2.68 in the first epoch to 0.68 by epoch 60, indicating the model successfully minimized the categorical cross-entropy loss and learned the statistical likelihood of character sequences. However, the qualitative results demonstrate that a well-trained model is not the only factor in producing human-readable text; the sampling strategy, controlled by the temperature hyperparameter ($\tau$), plays a critical role in controlling the entropy of the output distribution.

At a **temperature of 0.2**, the model exhibits extremely low randomness, effectively approximating a "greedy" sampling strategy. Mathematically, a low temperature sharpens the probability distribution output by the Softmax layer ($q_i = \frac{\exp(z_i/\tau)}{\sum \exp(z_j/\tau)}$), causing the model to overly favor the single most probable next character. In the early epochs (e.g., Epoch 1), this resulted in repetitive loops such as *"the wor the wor the perite."* By the later epochs (e.g., Epoch 10 and 48), the model learned words perfectly but fell into a deterministic trap, repeating phrases like *"the project gutenberg list the project gutenberg list"* or *"the companies and the companies."* This confirms that while low temperature maximizes local predictability, it fails to generate natural-sounding flow because it lacks the stochasticity required to transition between distinct semantic ideas.

At a **temperature of 0.5**, the results represent the optimal balance between coherence and creativity. By Epoch 30 and beyond, the model produces text that strongly resembles English, with proper sentence structure and very few spelling errors. For instance, in Epoch 46, it generated: *"bilities of development his one panacea for the ills of the companies to follow magame is cost of mexico."* While "magame" is likely a hallucination, the surrounding sentence structure is grammatically sound. Unlike the 0.2 setting, the 0.5 temperature introduces enough entropy to break the infinite repetition loops, allowing the Multinomial distribution to select slightly less probable—but still valid—characters. This setting most accurately reflects the style of the source text while maintaining readability.



At a **temperature of 1.0**, the model samples from the probability distribution without any re-weighting (standard Softmax). This creates a higher degree of unpredictability. While the text remains somewhat pronounceable, the coherence begins to degrade. In Epoch 60, the output included phrases like *"saceboraind in a diming seems as id a hibe those."* Here, we see the model inventing words (*"saceboraind," "hibe"*) that follow English phonetic rules but have no semantic meaning. This indicates that at a standard temperature, the model is confident enough to experiment with character combinations that statistically *look* correct but fail to form actual words.

Finally, at a **temperature of 1.2**, the text generation breaks down. This high temperature flattens the probability distribution toward a uniform distribution, increasing the entropy significantly. This causes the model to frequently select unlikely characters. Even at Epoch 60, where the model has a low loss value, the output at 1.2 contains strings like *"intabbero,"* *"cementermëal,"* and *"efigrates."* The syntax dissolves, and punctuation becomes erratic. This demonstrates that regardless of how well a model is trained, forcing high entropy during sampling destroys the learned relationships between characters.

In conclusion, the findings suggest that for character-level LSTM text generation, a moderate temperature (around 0.5) is required to produce high-quality results. It serves as a necessary filter to stabilize the model's predictions, preventing the robotic repetition of low temperatures and the chaotic hallucination of high temperatures.

***

### References

* **** Hochreiter, S., & Schmidhuber, J. (1997). "Long Short-Term Memory". *Neural Computation*, 9(8), 1735-1780.
* **** Goodfellow, I., Bengio, Y., & Courville, A. (2016). *Deep Learning*. MIT Press. (Chapters on Sequence Modeling and Output Layers).
* **** Hinton, G., Vinyals, O., & Dean, J. (2015). "Distilling the Knowledge in a Neural Network". *arXiv preprint arXiv:1503.02531*.
* **** Chollet, F. (2021). *Deep Learning with Python* (2nd Edition). Manning Publications. (Section 12.1 on Generating Text with LSTMs).
* **** Ackley, D. H., Hinton, G. E., & Sejnowski, T. J. (1985). "A learning algorithm for Boltzmann machines." *Cognitive Science*, 9(1), 147-169.